<a href="https://colab.research.google.com/github/pinilDissanayaka/Psychology-RAG-Fusion/blob/main/Notebook_RAG_Fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 19.2 MB/s eta 0:0

In [21]:
import os
from langchain_groq import ChatGroq
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_core.documents import Document

In [2]:
def getLlm():
  os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
  llm=ChatGroq(model_name='llama-3.1-70b-versatile',
              temperature=0.6)
  return llm

In [3]:
llm=getLlm()

In [4]:
os.environ['PINECONE_API_KEY']=userdata.get('PINECORN_API_KEY')
pinecone=Pinecone()

In [5]:
pineconeIndexNames=pinecone.list_indexes().names()

print('Available Indexes  :')
print(f'\t {pineconeIndexNames}')

Available Indexes  :
	 ['rag-fusion', 'multi-rag', 'constitution']


In [6]:
INDEX_NAME='rag-fusion'
DIMENSIONS=512

if not INDEX_NAME in pineconeIndexNames:
  pinecone.create_index(
          name=INDEX_NAME,
          dimension=DIMENSIONS,
          metric="euclidean",
          spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
          )
)
else:
  print("Index already exists")

Index already exists


In [7]:
pinecone.describe_index(INDEX_NAME)

{
    "name": "rag-fusion",
    "dimension": 512,
    "metric": "euclidean",
    "host": "rag-fusion-4myrn7y.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}

In [8]:
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question:

{question}"""


multiQyeryPrompt = ChatPromptTemplate.from_template(template)

print(multiQyeryPrompt)

input_variables=['question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is to generate five\ndifferent versions of the given user question to retrieve relevant documents from a vector\ndatabase. By generating multiple perspectives on the user question, your goal is to help\nthe user overcome some of the limitations of the distance-based similarity search.\nProvide these alternative questions separated by newlines. Original question:\n\n{question}'))]


In [9]:
def separateQuestions(question):
  return question.split('\n')

In [10]:
multiQyeryChain=multiQyeryPrompt | llm | StrOutputParser() | RunnableLambda(separateQuestions)

In [11]:
multiQyeryChain.invoke('how are you?')

["What's your status? ",
 'How do you feel today? ',
 "What's your current condition? ",
 "How's everything going? ",
 'Are you functioning properly?']

In [14]:
from urllib import request

file_name="Introducing Psychology"
url="https://ocw.mit.edu/ans7870/9/9.00SC/MIT9_00SCF11_text.pdf"

request.urlretrieve(url, file_name)

('Introducing Psychology', <http.client.HTTPMessage at 0x7b9d6b37ac20>)

In [18]:
loader=PyPDFLoader(file_name)

data=loader.load()

In [22]:
pageContent=list()

for page_content in data:
  pageContent.append(page_content.page_content)

print(f'Total pages: {len(pageContent)}')

Total pages: 783


In [24]:
model_name = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
chunker=SemanticChunker(embeddings)

In [ ]:
vectoreStore=PineconeVectorStore.from_documents(
    documents=data,
    embedding=embeddings,
    index_name=INDEX_NAME,
    text_key='text'
)

In [ ]:
retrever=vectoreStore.as_retriever()